In [1]:
import geopy
import psycopg2
from psycopg2 import sql, Error
from datetime import datetime
import requests
import pandas as pd
from geopy.distance import geodesic
import logging
import pytz
import threading
import time
import os



logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

            
def validate_and_convert_types(row):
    try:
        # Define the expected types for each column
        expected_types = {
            'COD': str,
            'Parada_Mais_Proxima': str,
            'REFRESH': pd.Timestamp,
            'LAT_IN_TIME': float,
            'LON_IN_TIME': float,
            'CODIGOLINHA': int,
            'TABELA': str,
            'SITUACAO': str,
            'SITUACAO2': str,
            'SENT': str,
            'SENTIDO_IN_TIME': str,
            'Hora': pd.Timestamp,
            'Distancia_ate_ponto': float,
            'SEQ': int,
            'SEQ_MAX': int

        }
        
        # Convert each column to the expected type
        for column, expected_type in expected_types.items():
            if column in row:
                if expected_type == pd.Timestamp:
                    row[column] = pd.to_datetime(row[column], errors='coerce')
                else:
                    row[column] = expected_type(row[column])

        return row
    except Exception as e:
        logging.error(f"Erro ao converter tipos de dados: {e}")
        return None

def inserir_bd(df_bus):
    conn = None
    try:
        df = df_bus.copy()
        if df['Sentido'].iloc[0] == 'sentido oposto':
            df['SEQ'] = df['NOVO_SEQ']

        conn_string = "host='localhost' dbname='Banco_IC' user='postgres' password='felipecs'"
        conn = psycopg2.connect(conn_string)
        with conn.cursor() as cur:
            for index, row in df.iterrows():
                # Validate and convert types
                row = validate_and_convert_types(row)
                if row is None:
                    continue  # Skip this row if type conversion failed

                insert_query = sql.SQL("""
                INSERT INTO data_onibus_sentido (COD, Parada_Mais_Proxima, REFRESH, LAT_IN_TIME, LON_IN_TIME, CODIGOLINHA, TABELA, SITUACAO, SITUACAO2, SENT, SENTIDO_IN_TIME, Hora, Distancia_ate_ponto, SEQ, SEQ_MAX)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                """)
                cur.execute(insert_query, (
                    row['COD'], 
                    row['Parada_Mais_Proxima'], 
                    row['REFRESH'], 
                    row['LAT_IN_TIME'], 
                    row['LON_IN_TIME'], 
                    row['CODIGOLINHA'], 
                    row['TABELA'], 
                    row['SITUACAO'], 
                    row['SITUACAO2'], 
                    row['SENT'], 
                    row['SENTIDO_IN_TIME'], 
                    row['Hora'], 
                    row['Distancia_ate_ponto'], 
                    row['SEQ'], 
                    row['SEQ_MAX'], 
                ))

            conn.commit()
            print("Dados inseridos com sucesso.")
    except (Exception, psycopg2.DatabaseError) as error:
        logging.error(f"Erro ao inserir dados: {error}")
    finally:
        if conn:
            conn.close()
def inverte_nome_ponto(df, df_paradas_linha_1, df_paradas_linha_2, df_paradas_linha_3):
    df_onibus = df.copy()

    if 'new_parada_Mais_Proxima' not in df.columns:
        df_onibus['new_parada_Mais_Proxima'] = ''
            
    if df_onibus['CODIGOLINHA'].iloc[0] == df_paradas_linha_1['COD'].iloc[0]:
        df1 = df_paradas_linha_1.copy()
    elif df_onibus['CODIGOLINHA'].iloc[0] == df_paradas_linha_2['COD'].iloc[0]:
        df1 = df_paradas_linha_2.copy()
    elif df_onibus['CODIGOLINHA'].iloc[0] == df_paradas_linha_3['COD'].iloc[0]:
        df1 = df_paradas_linha_3.copy()
    else:
        logging.error("Linha de Ã´nibus nÃ£o encontrada nas paradas fornecidas.")
        return None
    
    mapeamento = dict(zip(df1['SEQ'], df1['Nome']))
    df_onibus['new_parada_Mais_Proxima'] = df_onibus['NOVO_SEQ'].apply(lambda x: mapeamento.get(x, 'Parada nÃ£o encontrada'))
    
    return df_onibus

def inverte_sequencia(data_bus):
    df = data_bus.copy()
    df = df.reset_index(drop=True)
    
    if 'NOVO_SEQ' not in df.columns:
        df['NOVO_SEQ'] = 0

    def retorna_novo_seq(df, index):
        maior_seq = df['SEQ_MAX'].iloc[0]
        seq_atual = df['SEQ'].iloc[index]
        novo_seq = (maior_seq - seq_atual) + 1
        return novo_seq

    for index in range(df.shape[0]):
        if df['Sentido'].iloc[index] == 'sentido oposto':
            df.at[index, 'NOVO_SEQ'] = retorna_novo_seq(df, index)
    
    return df

def mandar_bd(df, df_paradas_linha_1, df_paradas_linha_2, df_paradas_linha_3):
    try:
        df_copy = df.copy()
        # Initialize FLAG_CICLO if it does not exist
        if 'FLAG_CICLO' not in df_copy.columns:
            df_copy['FLAG_CICLO'] = False  # Initialize as boolean
            
        if 'NOVO_SEQ' not in df_copy.columns:
            df_copy['NOVO_SEQ'] = 0

        # Set FLAG_CICLO for SEQ == SEQ_MAX
        df_copy['FLAG_CICLO'] = df_copy['SEQ'] == df_copy['SEQ_MAX']
        df_copy.loc[(df_copy['SEQ'] == df_copy['SEQ_MAX']) & (df_copy['Sentido'] == 'sentido certo'), 'FLAG_CICLO'] = True
        df_copy.loc[(df_copy['SEQ'] == 1) & (df_copy['Sentido'] == 'sentido oposto'), 'FLAG_CICLO'] = True
        
        # Remove duplicates based on specific columns and reset index
        df = df_copy.drop_duplicates(subset=['COD', 'Parada_Mais_Proxima', 'Distancia_ate_ponto']).reset_index(drop=True)
        
        # List to store codes for removal
        codigos_para_remover = []

        # Iterate through the DataFrame
        for index in range(len(df)):
            cod_in = df['COD'].iloc[index]
            flag = df['FLAG_CICLO'].iloc[index]
 
            if flag:
                cod_onibus_ciclo_fechado = cod_in
                df_bd = df[df['COD'] == cod_onibus_ciclo_fechado].copy()
                
                if df_bd['Sentido'].iloc[0] == 'sentido oposto':
                    df_sequencia_invertida = inverte_sequencia(df_bd)
                    if df_sequencia_invertida is None:
                        logging.error("Erro ao inverter ponto para o COD: %s", cod_onibus_ciclo_fechado)
                        continue
      
                    inserir_bd(df_sequencia_invertida)
                elif df_bd['Sentido'].iloc[0] == 'sentido certo':

                    inserir_bd(df_bd)
                    
                # Mark the code for removal
                codigos_para_remover.append(cod_onibus_ciclo_fechado)

        # Remove processed codes from DataFrame
        df = df[~df['COD'].isin(codigos_para_remover)]
        
        return df
    
    except Exception as e:
        logging.error("Erro no processamento em mandar_bd: %s", e)
        return None

def retorna_df_pontos_linha(linha):
    url_base = 'https://transporteservico.urbs.curitiba.pr.gov.br/getPontosLinha.php?linha={}&c=821f0'
    url = url_base.format(linha)
    try:
        response = requests.get(url)
        response.raise_for_status()
        dados_json = response.json()
        data = {
           
            'Nome': [item['NOME'] for item in dados_json],
            'NÃºmero': [item['NUM'] for item in dados_json],
            'Latitude': [float(item['LAT'].replace(',', '.')) for item in dados_json],
            'Longitude': [float(item['LON'].replace(',', '.')) for item in dados_json],
            'SEQ': [int(item['SEQ']) for item in dados_json],
            'GRUPO': [item['GRUPO'] for item in dados_json],
            'Sentido': [item['SENTIDO'] for item in dados_json],
            'Tipo': [item['TIPO'] for item in dados_json],
            'ID do ItinerÃ¡rio': [int(item['ITINERARY_ID']) for item in dados_json]
        }
        df1 = pd.DataFrame(data)
        df1['COD'] = linha
        sentido_a = df1['Sentido'].iloc[0]
        df_sentido_a = df1[df1['Sentido'] == sentido_a]

    except requests.RequestException as e:
        logging.error("Falha ao acessar a pÃ¡gina: %s", e)
        return None, None



    return df_sentido_a

def calcular_distancia_ponto(df, df_pontos):
    try:
        distancias = []
        for index, row in df.iterrows():
            lat_onibus = row['LAT_IN_TIME']
            lon_onibus = row['LON_IN_TIME']
            ponto_mais_proximo = row['Parada_Mais_Proxima']
            ponto = df_pontos[df_pontos['Nome'] == ponto_mais_proximo].iloc[0]
            distancia = calcular_distancia(lat_onibus, lon_onibus, ponto['Latitude'], ponto['Longitude'])
            distancias.append(distancia)
        return distancias
    except Exception as e:
        logging.error("Erro ao calcular distÃ¢ncia atÃ© o ponto mais prÃ³ximo: %s", e)
        return None

def calcular_distancia(lat1, lon1, lat2, lon2):
    try:
        coords_1 = (lat1, lon1)
        coords_2 = (lat2, lon2)
        distancia = geopy.distance.geodesic(coords_1, coords_2).meters
        return distancia
    except Exception as e:
        logging.error("Erro ao calcular distÃ¢ncia: %s", e)
        return None
    
def encontrar_parada_mais_proxima(lat, lon, df_paradas):
    try:
        if df_paradas.empty:
            logging.warning("O DataFrame df_paradas estÃ¡ vazio.")
            return None



        distancias = df_paradas.apply(lambda row: calcular_distancia(lat, lon, row['Latitude'], row['Longitude']), axis=1)
        index_parada_mais_proxima = distancias.idxmin()
        nome_parada_mais_proxima = df_paradas.loc[index_parada_mais_proxima, 'Nome']

        return nome_parada_mais_proxima
    except KeyError as ke:
        logging.error("Chave nÃ£o encontrada: %s", ke)
        return None
    except Exception as e:
        logging.error("Erro ao encontrar parada mais prÃ³xima: %s", e)
        return None



def manter_menor_distancia(df):
    try:
        # Aplica a funcao nsmallest para obter a menor distÃ¢ncia em cada grupo
        df_menor_distancia = df.groupby(['COD', 'Parada_Mais_Proxima']).apply(lambda x: x.nsmallest(1, 'Distancia_ate_ponto')).reset_index(drop=True)
        return df_menor_distancia
    except Exception as e:
        logging.error("Erro ao manter menor distÃ¢ncia: %s", e)
        return df


def retorna_sentido(df):

    df_copy = df.copy()
    df_copy.sort_values(by=['COD', 'Hora'], inplace=True)

    # Adiciona a coluna 'Sentido' se naoexistir
    if 'Sentido' not in df_copy.columns:
        df_copy['Sentido'] = ''

    # Obtem os cÃ³digos Ãºnicos
    codigos_unicos = df_copy['COD'].unique()

    # Itera sobre cada codigo unico
    for cod in codigos_unicos:
        df_cod = df_copy[df_copy['COD'] == cod].reset_index(drop=True)

        # Verifica o sentido da sequencia
        sentido = ''
        for i in range(len(df_cod) - 1):
            if df_cod['SEQ'].iloc[i] < df_cod['SEQ'].iloc[i + 1]:
                sentido = 'sentido certo'
            elif df_cod['SEQ'].iloc[i] > df_cod['SEQ'].iloc[i + 1]:
                sentido = 'sentido oposto'
            else:
                sentido = 'onibus fora de tabela'


        # Aplica o sentido calculado
        df_copy.loc[df_copy['COD'] == cod, 'Sentido'] = sentido

    return df_copy
    
def calcular_sentido_certo(df):
    
    df_codigos_onibus_sentido = df.copy()
    for i in range(len(df_codigos_onibus_sentido)):
            if(df_codigos_onibus_sentido['Sentido'].iloc[i] == 'sentido oposto'):
                df_codigos_onibus_sentido['SEQ'].iloc[i] =(df_codigos_onibus_sentido['SEQ_MAX'].iloc[i] - df_codigos_onibus_sentido['SEQ'].iloc[i]) + 1
    
    return df_codigos_onibus_sentido
                
def retorna_seq_max(df, df_paradas):
    # Copia o dataframe para evitar mudanÃ§as no original
    df = df.copy()

    # Converte a coluna 'CODIGOLINHA' para int
    df['CODIGOLINHA'] = df['CODIGOLINHA'].astype(int)

    # Adiciona a coluna 'SEQ_MAX' se nÃ£o existir
    if 'SEQ_MAX' not in df.columns:
        df['SEQ_MAX'] = 0

    # Cria um dicionario de mapeamento de COD para o maior valor de SEQ
    mapeamento_seq_max = df_paradas.groupby('COD')['SEQ'].max().to_dict()

    # Aplica o mapeamento para obter SEQ_MAX
    df['SEQ_MAX'] = df['CODIGOLINHA'].apply(lambda x: mapeamento_seq_max.get(x, 0))

    return df

                
def buscar_e_processar_dados(linha, df_paradas_linha_1, df_paradas_linha_2, df_paradas_linha_3):

    url_base = 'https://transporteservico.urbs.curitiba.pr.gov.br/getVeiculos.php?linha={}&c=821f0'.format(linha)
    try:
       
        response = requests.get(url_base)
        response.raise_for_status()

        timezone_sp = pytz.timezone('America/Sao_Paulo')
        hora_online = datetime.now(timezone_sp).strftime("%Y-%m-%d %H:%M:%S")

        dados_json = response.json()
        codigos_onibus = [{
            "COD": valor['COD'],
            "REFRESH": valor['REFRESH'],
            "LAT_IN_TIME": valor['LAT'],
            "LON_IN_TIME": valor['LON'],
            "CODIGOLINHA": valor['CODIGOLINHA'],
            "TABELA": valor['TABELA'],
            "SITUACAO": valor['SITUACAO'],
            "SITUACAO2": valor['SITUACAO2'],
            "SENT": valor['SENT'],
            "SENTIDO_IN_TIME": valor['SENTIDO']
        } for chave, valor in dados_json.items()]

        df_codigos_onibus = pd.DataFrame(codigos_onibus)
        
        #remover linhas que nÃ£o estÃ£o em operaÃ§Ã£o, mas aparecem no site
        indices_para_remover = df_codigos_onibus[df_codigos_onibus['SITUACAO2'] != 'REALIZANDO ROTA'].index
        # Removendo as linhas pelos Ã­ndices
        
        df_codigos_onibus.drop(index=indices_para_remover, inplace=True)
        
        

        df_codigos_onibus.reset_index(drop=True, inplace=True)
        
        if(df_paradas_linha_1['COD'].iloc[0] == linha):
            
            df_sentido_pontos = df_paradas_linha_1.copy()

            
        elif(df_paradas_linha_2['COD'].iloc[0] == linha):
            
            df_sentido_pontos = df_paradas_linha_2.copy()


            
        elif(df_paradas_linha_3['COD'].iloc[0] == linha):
            
            df_sentido_pontos = df_paradas_linha_3.copy()

            
        else:
            logging.info("ERRO, DATFRAME DA LINHA NÃO INSERIDO")
            

        df_codigos_onibus['Hora'] = hora_online
        mapeamento = dict(zip(df_sentido_pontos['Nome'], df_sentido_pontos['SEQ']))
    

        # Encontrar parada mais proxima vetorizado
        df_codigos_onibus['Parada_Mais_Proxima'] = df_codigos_onibus.apply(
            lambda row: encontrar_parada_mais_proxima(row['LAT_IN_TIME'], row['LON_IN_TIME'], df_sentido_pontos),
            axis=1
        )

        
        df_codigos_onibus['Distancia_ate_ponto'] = calcular_distancia_ponto(df_codigos_onibus, df_sentido_pontos)
        
        df_codigos_onibus['SEQ'] = df_codigos_onibus['Parada_Mais_Proxima'].apply(lambda x: mapeamento.get(x))

        df_codigos_onibus_max = retorna_seq_max(df_codigos_onibus,df_sentido_pontos)
        

       
        return df_codigos_onibus_max
    
    except requests.RequestException as e:
        logging.error("Falha ao acessar a pÃ¡gina: %s", e)
        return None
    except Exception as e:
        logging.error("Erro ao processar dados: %s", e)
        return None
    

    
################################################################################################################################

linhas = [203, 503, 303]

df_paradas_linha_1 = retorna_df_pontos_linha(203)
df_paradas_linha_2 = retorna_df_pontos_linha(503)
df_paradas_linha_3 = retorna_df_pontos_linha(303)

df_concatenado = pd.DataFrame()

def processar_linha(linha, result_list):
    try:
        df_result = buscar_e_processar_dados(linha, df_paradas_linha_1, df_paradas_linha_2, df_paradas_linha_3)
        if df_result is not None and not df_result.empty:
            result_list.append(df_result)
    except Exception as e:
        logging.error(f"Erro ao processar linha {linha}: {e}")

def dentro_do_horario():
    agora = datetime.now().time()
    inicio = datetime.strptime("05:30", "%H:%M").time()
    fim = datetime.strptime("23:30", "%H:%M").time()
    return inicio <= agora <= fim





# Loop principal
while True:
    if dentro_do_horario():
        result_list = []
        threads = [threading.Thread(target=processar_linha, args=(linha, result_list)) for linha in linhas]

        for thread in threads:
            thread.start()

        for thread in threads:
            thread.join()

        if result_list:
            for df_result in result_list:
                df_concatenado = pd.concat([df_concatenado, df_result], ignore_index=True)
            df_concatenado = df_concatenado.drop_duplicates(['COD', 'REFRESH']).reset_index(drop=True)
            df_concatenado = manter_menor_distancia(df_concatenado)
            df_concatenado = retorna_sentido(df_concatenado)
            df_concatenado = mandar_bd(df_concatenado, df_paradas_linha_1, df_paradas_linha_2, df_paradas_linha_3)




        # Espera 60 segundos antes da próxima iteração
        time.sleep(60)

KeyboardInterrupt: 